# Image classifier using PyTorch

In [ ]:
# Get data
# Build model
# Train model
# Test model

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# Get train data

In [ ]:
# With transform
train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor())

In [ ]:
type(train_data[0][0])

In [ ]:
train_data[0][0].min()

In [ ]:
train_data[0][0].max()

In [ ]:
train_data[0][0].dtype

In [ ]:
train_data[0][0].shape

In [ ]:
train_data[0][0][0][20]

# Get test data

In [ ]:
# With transform
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

In [ ]:
type(train_data[0][0])

In [ ]:
test_data[0][0].shape

In [ ]:
test_data[0][0][0][20]

In [ ]:
# Map label index to label name
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [ ]:
# Show samples
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

In [ ]:
# Dataloaders
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

In [ ]:
# Choose device
device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
          else "cpu")
print('DEVICE=', device)

# Layers used to build model

In [ ]:
# Data
X = torch.rand(1, 3, 2, device=device)
print(X)


In [ ]:
# Flatten
l = nn.Flatten()
X = l(X)
print(X)


In [ ]:
# Linear
l =  nn.Linear(in_features=3*2, out_features=4)
X = l(X)
print(X)


In [ ]:
# ReLU
l =  nn.ReLU()
X = l(X)
print(X)


In [ ]:
# Sequential
seq_modules = nn.Sequential(
    nn.Flatten(),
    nn.Linear(6, 4),
    nn.ReLU(),
    nn.Linear(4, 2)
)
input_image = torch.rand(1,3,2)
logits = seq_modules(input_image)
print('logits',logits.shape, logits)

In [ ]:
# Softmax
l =  nn.Softmax(dim=1)
pred_probability = l(logits)
print('pred_probability=', pred_probability)

# Build model

In [ ]:
# Create a class that inherits from nn.Module
# Define the layers of the network in the __init__ function
# Specify how data will pass through the network in the forward function
# To accelerate operations in the neural network, we move it to the GPU if available

In [ ]:
# Define neural network
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # Flattens input to vector
        self.linear_relu_stack = nn.Sequential(  # Applies sequence of Linear and ReLU layers
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = Net().to(device)

In [ ]:
model

In [ ]:
# Pass random input through model
X = torch.rand(1, 28, 28, device=device)
print('X=', type(X), X.shape, X.dtype)

logits = model(X)
print('logits=', type(logits), logits.shape, logits.dtype)

y_prob = nn.Softmax(dim=1)(logits)
print('y_prob=', type(y_prob), y_prob.shape, y_prob.dtype, y_prob)

y_pred = y_prob.argmax(1)
print(f"Predicted class: {y_pred}")

In [ ]:
# Params
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

# Loss function and Optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# Train model

In [ ]:
# In a single training loop, the model makes predictions on the training dataset (fed to it in batches), 
# and backpropagates the prediction error to adjust the model’s parameters.

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [ ]:
# We also check the model’s performance against the test dataset to ensure it is learning.

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# The training process is conducted over several iterations (epochs). 
# During each epoch, the model learns parameters to make better predictions. 
# We print the model’s accuracy and loss at each epoch
# we’d like to see the accuracy increase and the loss decrease with every epoch.

epochs = 2

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


# Save model

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

# Load model

In [ ]:
model = Net().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

# Make predictions

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')